In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import regex as re
from scrapy.selector import Selector
from scrapy.http import HtmlResponse

In [ ]:
# hyperlink = "https://archive.cnx.org/contents/havxkyvS@9.423:Gv1bKcKt@6/Chemistry-in-Context"
# name = requests.get(hyperlink)
# #soup = BeautifulSoup(name, 'lxml')

In [ ]:
# Turns JSON from https://archive.cnx.org/contents/85abf193-2bd2-4908-8563-90b8a7ac8df6%409.423.json to dictionary 

# with open('chem_directory') as f:
#         read_data = f.read()
# read_data=read_data.replace('null','None').replace('false','False')
# exec('dict='+read_data)

In [ ]:
# This code scraped html from the Open Stax pages and saved them to a
# dictionary. The dictionary has been pickled, so only use this code
# if pickled file needs to be replaced


# with open('chem_directory') as f:
#         read_data = f.read()
# read_data=read_data.replace('null','None').replace('false','False')
# exec('dict='+read_data)

# import time
# pages={}
# for chapter in dict['tree']['contents'][1:22]:
#     chapter_title=chapter['title']
#     pages[chapter_title]={}
#     for section in chapter['contents'][1:]:
#         #print(chapter_title)        
#         hyperlink='https://archive.cnx.org/contents/'+section['id']
#         name = requests.get(hyperlink)

#         section_title=section['title']
#         pages[chapter_title][section_title]= name.text
#         time.sleep(1)

# import pickle

# pickle.dump( pages, open( "open_stax.p", "wb" ) )


In [ ]:
import pickle

pages = pickle.load( open( "open_stax.p", "rb" ) )


In [ ]:
df=pd.DataFrame.from_dict({(i,j): pages[i][j] 
                           for i in pages.keys() 
                           for j in pages[i].keys()},
                       orient='index')


In [ ]:
df['label']=df.index
df['chapter']=df['label'].apply(lambda x: x[0])
df['section']=df['label'].apply(lambda x: x[1])

In [ ]:
df.head()

In [ ]:
seed_df=pd.DataFrame([['a','a','a'],['a','a','a'],['a','a','a']])

In [ ]:
# This code cleaned up html text and put in dataframe
# for i in range(150):
#     try:
#         tsoup= BeautifulSoup(df[0][i], 'lxml')
#         html=Selector(text=tsoup.prettify()).xpath("//p").extract()
#         for entry in html:
#             paragraph=entry.replace("\n   ",'').replace("\n","").replace(' <sup>  ',"").replace(' </sup> ','').replace(' <sub>  ',"").replace(' </sub> ','')
#             seed_df=seed_df.append([[df.iloc[i,2],df.iloc[i,3],paragraph]])
#     except:
#         break

In [ ]:
seed_df.columns=['Chapter','Section','Text']

In [ ]:
seed_df.index=list(range(seed_df.shape[0]))

In [ ]:
seed_df.head()

In [ ]:
seed_df=seed_df[3:]

In [ ]:
# import pickle

# pickle.dump( seed_df, open( "open_stax_dataset.p", "wb" ) )

In [ ]:
import pickle

seed_df = pickle.load( open( "open_stax_dataset.p", "rb" ) )


In [ ]:
seed_df.shape

In [ ]:
def remove_punctuation(x):
    import re
    x=re.sub(r'<.*?>','',x)
    table= str.maketrans(dict.fromkeys('.,?!();:'))
    return x.translate(table)
def stem_the_text(x):
    text_list=x.split()
    new_text=''
    from nltk.stem import PorterStemmer
    stemmer = PorterStemmer()
    for item in text_list:
        if len(item)>3:
            new_text +=stemmer.stem(item)+' '
        else:
            new_text +=item+' '
    return new_text
def rem_stem(x):
    return stem_the_text(remove_punctuation(x))

In [ ]:
seed_df['Stemmed']=seed_df['Text'].apply(rem_stem)

In [ ]:
seed_df=seed_df[~seed_df['Stemmed'].duplicated(keep=False)]

In [ ]:
# import pickle

# pickle.dump( seed_df, open( "stemmed_df.p", "wb" ) )

In [2]:
import pickle

seed_df=pickle.load( open( "stemmed_df.p", "rb" ) )

In [3]:
def get_word_count(x):
    return len(x.split())

In [4]:
seed_df=seed_df[seed_df['Stemmed'].apply(get_word_count)>10]

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(min_df=6, max_df=.18, ngram_range=(1,4),binary=True)
cv.fit(seed_df['Stemmed'])
soil=cv.transform(seed_df['Stemmed'])
len(cv.get_feature_names())

14241

In [6]:
cv.get_feature_names()[318:]

['ab',
 'abbrevi',
 'abil',
 'abil of',
 'abil to',
 'abl',
 'abl to',
 'about',
 'about 10',
 'about 20',
 'about 25',
 'about of',
 'about of the',
 'about one',
 'about the',
 'abov',
 'abov the',
 'absenc',
 'absenc of',
 'absenc of air',
 'absolut',
 'absolut temperatur',
 'absorb',
 'absorb by',
 'absorb by the',
 'absorb heat',
 'absorb or',
 'absorpt',
 'absorpt of',
 'abstract',
 'abund',
 'abund of',
 'acceler',
 'accept',
 'accept proton',
 'acceptor',
 'accommod',
 'accompani',
 'accompani by',
 'accompani the',
 'accomplish',
 'accord',
 'accord to',
 'accord to the',
 'accord to the equat',
 'accord to the follow',
 'accord to thi',
 'accord to thi equat',
 'accord with',
 'account',
 'account for',
 'account for the',
 'accumul',
 'accur',
 'accuraci',
 'acet',
 'acet acid',
 'acet acid and',
 'acet acid ch',
 'acet acid ch co',
 'acet ion',
 'acetylen',
 'achiev',
 'achiev the',
 'acid',
 'acid and',
 'acid and base',
 'acid and its',
 'acid and the',
 'acid are',
 'aci

In [ ]:
# stems={}
# for word in cv.get_feature_names()[5000:5002]:
#     name0='stems'
#     for number in range(1,min(7,len(word)+1)):
#         exec('name{}'.format(number)+'=name{}'.format(number-1)+'+"['+"'" +'{}'.format(word[:number])+"'"+']'+'"')
#       #  'name{}'.format(number)

In [7]:
soil=pd.DataFrame(soil.todense(),columns=cv.get_feature_names())

In [8]:
soil=soil[cv.get_feature_names()[318:]]

In [ ]:
import pickle

pickle.dump( soil, open( "count_vectorized_df.p", "wb" ) )

In [ ]:
import pickle

soil=pickle.load( open( "count_vectorized_df.p", "rb" ) )

In [ ]:
# from sklearn.decomposition import PCA, TruncatedSVD
# # svd=TruncatedSVD(20)
# # svd.fit(soil)
# # svd_soil=svd.transform(soil)

# pca=PCA()
# pca.fit(soil)
# #pca_soil=pca.transform(soil)

In [ ]:
soil.shape

In [ ]:
from sklearn.decomposition import TruncatedSVD
pca=TruncatedSVD(500)
pca.fit(soil.as_matrix())


In [ ]:
pca.explained_variance_ratio_.sum()

In [9]:
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(soil.as_matrix(), seed_df['Chapter'])


In [11]:
from sklearn.naive_bayes import BernoulliNB, MultinomialNB, GaussianNB
mnb=MultinomialNB()
mnb.fit(X_train, y_train)
print(mnb.score(X_test, y_test))
#print(bnb.score(soil_test, y_test))
#print(confusion_matrix(seed_df['Section'],bnb.predict(soil)))
#print(classification_report(y_test,bnb.predict(soil)))

0.712581344902


In [ ]:
print(confusion_matrix(seed_df['Chapter'],mnb.predict(soil)))
print(classification_report(y_test,mnb.predict(X_test)))

In [10]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()
lr.fit(X_train, y_train)
print(lr.score(X_test, y_test))
#print(bnb.score(soil_test, y_test))
#print(confusion_matrix(seed_df['Chapter'],lr.predict(soil)))
#print(classification_report(y_test,lr.predict(X_test)))

0.73535791757


In [ ]:
# from sklearn.model_selection import GridSearchCV
# from sklearn.linear_model import LogisticRegression
# parameters = {'class_weight':('balanced', None), 'solver':('newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga')}
# lr_for_gs=LogisticRegression()
# lrgs = GridSearchCV(lr_for_gs, parameters,verbose=2)
# lrgs.fit(soil.as_matrix(), seed_df['Section'])

In [ ]:
print(confusion_matrix(seed_df['Chapter'],lr.predict(soil)))
print(classification_report(y_test,lr.predict(X_test)))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
rf=RandomForestClassifier(200)
rf.fit(X_train, y_train)
print(rf.score(X_test, y_test))
#print(bnb.score(soil_test, y_test))
#print(confusion_matrix(seed_df['Chapter'],lr.predict(soil)))
#print(classification_report(y_test,lr.predict(X_test)))

In [ ]:
print(confusion_matrix(seed_df['Chapter'],rf.predict(soil)))
print(classification_report(y_test,rf.predict(X_test)))

In [ ]:
from sklearn.ensemble import VotingClassifier
vc = VotingClassifier(estimators=[
        ('lr', lr), ('rf', rf), ('mnb', mnb)], voting='soft')

vc.fit(X_train, y_train)
print(vc.score(X_test, y_test))




In [ ]:
print(confusion_matrix(seed_df['Chapter'],vc.predict(soil)))
print(classification_report(y_test,vc.predict(X_test)))

In [ ]:
sanity_check=pd.DataFrame()

In [ ]:
print

In [ ]:
sanity_check['label']=seed_df['Section']
#sanity_check['lr_prediction']=lr.predict(soil)
#sanity_check['rf_prediction']=rf.predict(soil)
#sanity_check['mnb_prediction']=mnb.predict(soil)
sanity_check['vc_prediction']=vc.predict(soil)
sanity_check['original_text']=seed_df['Text']
sanity_check['stemmed_text']=seed_df['Stemmed']

In [ ]:
def is_right(x):
    if x==True:
        return 1
    else:
        return 0

In [ ]:
sanity_check['lr_correct']=sanity_check['label']==sanity_check['lr_prediction']
sanity_check['rf_correct']=sanity_check['label']==sanity_check['rf_prediction']
sanity_check['mnb_correct']=sanity_check['label']==sanity_check['mnb_prediction']
sanity_check['vc_correct']=sanity_check['label']==sanity_check['vc_prediction']

In [ ]:
sanity_check['lr_correct_num']=sanity_check['lr_correct'].apply(is_right)
sanity_check['rf_correct_num']=sanity_check['rf_correct'].apply(is_right)
sanity_check['mnb_correct_num']=sanity_check['mnb_correct'].apply(is_right)
sanity_check['vc_correct_num']=sanity_check['vc_correct'].apply(is_right)

In [ ]:
sanity_check.head()

In [ ]:
sanity_check['number_correct']=sanity_check['lr_correct_num']+sanity_check['rf_correct_num']+sanity_check['mnb_correct_num']


In [ ]:
sanity_check[sanity_check['vc_correct']].shape

In [ ]:
sanity_check['word_count']=seed_df['Text'].apply(get_word_count)
sanity_check['stemmed_word_count']=seed_df['Stemmed'].apply(get_word_count)

In [ ]:
sanity_check[sanity_check['stemmed_word_count']<10].shape

In [ ]:
sanity_check[(sanity_check['label']!=sanity_check['prediction'])].shape

In [ ]:
num=113
print(sanity_check[(sanity_check['label']!=sanity_check['prediction'])].iloc[num])
print(sanity_check[(sanity_check['label']!=sanity_check['prediction'])].iloc[num]['original_text'])



In [ ]:
sanity_check[sanity_check['label']!=sanity_check['prediction']].describe()

In [ ]:
sanity_check[sanity_check['label']==sanity_check['prediction']].describe()

In [ ]:
sanity_check[sanity_check['label']!=sanity_check['prediction']]['stemmed_text'].duplicated().value_counts()

In [ ]:
sanity_check[sanity_check['label']==sanity_check['prediction']]['stemmed_text'].duplicated().value_counts()

In [ ]:
sanity_check[sanity_check['word_count']>160].iloc[0]
sanity_check[sanity_check['word_count']<10].iloc[]['original_text']

In [ ]:
question='The gastric juice of a person contains 2.19g of HCl per litre how many antacid tablets each containing 520mg of Al(OH)3 are required to neutralize the acid completely'
#question='The radius of a silver atom is 145 pm. How many silver atoms would have to be laid side by side to span a distance of 4.70 mm?'
#question='A block of wood 3.0cm on each side has a mass of 27g . What is the density of this block ?'
#question='How do you calculate the ideal gas law constant?'
question='When are roman numerals used when naming compounds'
#question='Osmium has a density of 22.6 #"g/cm"^3#. What volume (in #"cm"^3#) would be occupied by a 0.0218 kg sample of osmium. Is it easier to do the V= m/d formula of should you do a conversion since you have kg and #"cm"^3#?'
#question='Why is water liquid where as H2S is gas at room temperature?'
#question='Why are gamma rays harmful to living creatures?'
#question='How is molecular polarity related to solubility?'
question='What is the enthalpy of formation of carbon monoxide in KJ/mol ? C(s) + O2(g) --> CO2(g) ΔH° = -393 kJ 2CO(g) + O2(g) --> 2CO(g) ΔH° = -588 kJ'
question='What physical or chemical changes occur in a flame?'

In [ ]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
new_question=''    
for item in question.split():
    if len(item)>3:
        new_question +=stemmer.stem(item)+' '
    else:
        new_question +=item+' '

In [ ]:
stemmer.stem('boric')

In [ ]:
cv.get_feature_names()[497:]

In [ ]:
testpd=cv.transform([new_question, 'testing'])

In [ ]:
testpd.shape

In [ ]:
vc.predict(testpd[:,497:])

In [ ]:
vc.predict_proba(testpd[:,497:])

In [ ]:
for group,prob in zip(vc.classes_,vc.predict_proba(testpd[:,497:])[0]):
    print(group,prob)

In [ ]:
len(vc.classes_)